<a href="https://colab.research.google.com/github/vector777-ai/DeepSpaceExoplanetHunting/blob/master/DeepSpaceExoplanetSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive

/content/drive


In [0]:
cd My Drive

/content/drive/My Drive


In [0]:
cd ExoPlanetData

/content/drive/My Drive/ExoPlanetData


In [0]:
import numpy as np
import pandas as pd

In [0]:
def initialize_parameters(layer_dims):
    parameters={}
    L=len(layer_dims)
    for l in range(1,L):
        parameters["W"+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters["b"+str(l)]=np.random.randn(layer_dims[l],1)*0.01
    return parameters

In [0]:
def linear_forward(A_prev,W,b):
    Z=np.dot(W,A_prev)+b
    cache=(A_prev,W,b)
    return Z,cache

In [0]:
def linear_activation_forward(A_prev,W,b,activation):
    (activation_cache,linear_cache)=linear_forward(A_prev,W,b)
    if activation == "tanh":
        A=np.tanh(activation_cache)
    elif activation == "sigmoid":
        A=1/(1+np.exp(-activation_cache))
    cache=(linear_cache,activation_cache)
    return A,cache

In [0]:
def Linear_model_forward(X,parameters):
    L=len(parameters)//2
    caches=[]
    A=X
    for l in range(1,L):
        A_prev=A
        A,cache=linear_activation_forward(A_prev,parameters["W"+str(l)],parameters["b"+str(l)],"tanh")
        caches.append(cache)
    A,cache=linear_activation_forward(A,parameters["W"+str(L)],parameters["b"+str(L)],"sigmoid")
    caches.append(cache)
    return A,caches

In [0]:
def compute_cost(Y,AL):
    m=Y.shape[1]
    cost=-np.sum((Y*(np.log(AL)))+((1-Y)*np.log(1-AL)))/m
    cost=np.squeeze(cost)
    return cost

In [0]:
def linear_backward(dZ,cache):
    A_prev,W,b=cache
    m=A_prev.shape[1]
    dW=np.dot(dZ,A_prev.T)/m
    db=np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev=np.dot(W.T,dZ)
    return dA_prev,dW,db

In [0]:
def linear_activation_backward(dA,cache,activation):
    (linear_cache,activation_cache)=cache
    Z=activation_cache
    if activation == "tanh":
        dZ=1-(np.tanh(Z)**2)
    elif activation == "sigmoid":
        dZ=((dA)*(np.exp(-Z)/((1+np.exp(-Z))**2)))
    dA_prev,dW,db=linear_backward(dZ,linear_cache)
    return dA_prev,dW,db

In [0]:
def Linear_model_backward(Y,AL,parameters,caches):
    m=Y.shape[1]
    L=len(parameters)//2
    grads={}
    dAL=-(np.divide(Y,AL)-np.divide((1-Y),(1-AL)))
    curr_cache=caches[L-1]
    grads["dA"+str(L-1)],grads["dW"+str(L)],grads["db"+str(L)]=linear_activation_backward(dAL,curr_cache,"sigmoid")
    for l in reversed(range(L-1)):
        curr_cache=caches[l]
        grads["dA"+str(l)],grads["dW"+str(l+1)],grads["db"+str(l+1)]=linear_activation_backward(grads["dA"+str(l+1)],curr_cache,"tanh")
    return grads

In [0]:
def update_parameters(parameters,grads,learning_rate=0.01):
    L=len(parameters)//2
    for l in range(1,L+1):
        parameters["W"+str(l)]=parameters["W"+str(l)]-(learning_rate*grads["dW"+str(l)])
        parameters["b"+str(l)]=parameters["b"+str(l)]-(learning_rate*grads["db"+str(l)])
    return parameters

In [0]:
def compute_accuracy(Y,AL):
    m=Y.shape[1]
    AL_temp=1.0*(AL>=0.5)
    diff=1.0*(Y==AL_temp)
    acc=np.sum(diff)/m
    acc=np.squeeze(acc)
    return acc

In [0]:
def Model():
    #Data Load
    train_data=pd.read_csv("./exoTrain.csv")
  
    Y_train=np.zeros((1,5087))
    Y_train[0]=train_data["LABEL"].T-1
  
    train_data=train_data.drop("LABEL",axis=1)
  
    X_train=np.zeros((3197,5087))
    X_train=train_data.as_matrix().T
    
    #Hyperparameter tuning
    layer_dims=[3197,64,64,1]
    learning_rate=0.01
    epoches=1400
    
    parameters=initialize_parameters(layer_dims)
    for e in range(1,epoches+1):
        AL,caches=Linear_model_forward(X_train,parameters)
        cost=compute_cost(Y_train,AL)
        if e%100 == 0:
            acc=compute_accuracy(Y_train,AL)
            print("Cost at epoch "+str(e)+": "+str(cost)+"\t=========\tAcc: "+str(acc))
        grads=Linear_model_backward(Y_train,AL,parameters,caches)
        parameters=update_parameters(parameters,grads,learning_rate)
    train_acc=compute_accuracy(Y_train,AL)
    print("Training Accuracy: "+str(train_acc))
    return parameters

In [0]:
def predict():
    parameters=Model()
    test_data=pd.read_csv("./exoTest.csv")
  
    Y_test=np.zeros((1,570))
    Y_test[0]=test_data["LABEL"].T-1
  
    test_data=test_data.drop("LABEL",axis=1)
  
    X_test=np.zeros((3197,570))
    X_test=test_data.as_matrix().T
    Y_hat,caches=Linear_model_forward(X_test,parameters)
    test_acc=compute_accuracy(Y_test,Y_hat)
    return test_acc

In [0]:
test_acc=predict()
print("Test Accuracy: "+str(test_acc))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


Cost at epoch 100: 0.2743854669161055	=========	Acc: 0.8954196972675447
Cost at epoch 200: 0.22677202806794272	=========	Acc: 0.9131118537448398
Cost at epoch 300: 0.19646323483868128	=========	Acc: 0.9225476705327305
Cost at epoch 400: 0.17233380628750405	=========	Acc: 0.931393748771378
Cost at epoch 500: 0.15415866257359556	=========	Acc: 0.9380774523294673
Cost at epoch 600: 0.13915386021825912	=========	Acc: 0.942991940239827
Cost at epoch 700: 0.12215447321318716	=========	Acc: 0.9484961666994299
Cost at epoch 800: 0.10338023279276211	=========	Acc: 0.9549832907411048
Cost at epoch 900: 0.08010759935137986	=========	Acc: 0.9644191075289955
Cost at epoch 1000: 0.05679411044842738	=========	Acc: 0.9919402398270101
Cost at epoch 1100: 0.0463408543452788	=========	Acc: 0.9927265578926676
Cost at epoch 1200: 0.04244167495381726	=========	Acc: 0.9927265578926676
Cost at epoch 1300: 0.04122213417122483	=========	Acc: 0.9927265578926676
Cost at epoch 1400: 0.04090588173818346	=========	A